In [30]:
import cv2 as cv
import mediapipe as mp
import time
import math
import numpy as np
import datetime
import cv2.aruco as aruco


class poseDetector():
    
    
    def __init__(self, mode=False, upBody=False, smooth=True, detectionCon=0.8, trackCon=0.8):
        self.mode = mode
        self.upBody = upBody
        self.smooth = smooth
        self.detectionCon = detectionCon
        self.trackCon = trackCon
        
        self.mpDraw = mp.solutions.drawing_utils
        self.mpPose = mp.solutions.pose
        self.pose = self.mpPose.Pose(self.mode, self.upBody, self.smooth,
                                    self.detectionCon, self.trackCon)
        
    def findPose(self, img, draw=False):
        imgRGB = cv.cvtColor(img, cv.COLOR_BGR2RGB)
        self.results = self.pose.process(imgRGB)
        if self.results.pose_landmarks:
            if draw:
                self.mpDraw.draw_landmarks(img, self.results.pose_landmarks, 
                                          self.mpPose.POSE_CONNECTIONS)
        return img
    
    
    def findPosition(self, img, draw=False):
        self.lmList = []
        if self.results.pose_landmarks:
            for id, lm in enumerate(self.results.pose_landmarks.landmark):
                h, w, c = img.shape
#                 print(id, lm)
                cx, cy = int(lm.x*w), int(lm.y*h)
                self.lmList.append([id, cx, cy])
#                 if draw:
#                     cv.circle(img, (cx,cy), 5, (255,0,0), cv.FILLED)
        return self.lmList


    def findAngle(self, img, p1, p2, p3, pp1, pp2, pp3, draw=True):
        if len(self.lmList) > 26:
            x1,y1 = self.lmList[p1][1:]
            x2,y2 = self.lmList[p2][1:]
            x3,y3 = self.lmList[p3][1:]
            
#           calculo del angulo
            v0 = np.array([x1,y1]) - np.array([x2,y2])
            v1 = np.array([x3,y3]) - np.array([x2,y2])
    
            angle = abs(np.degrees(np.math.atan2(np.linalg.det([v0,v1]), np.dot(v0 ,v1))))
#             print(angle)

#           Evitar saltos
#           puntos aux        
            xx1,yy1 = pp1
            xx2,yy2 = pp2
            xx3,yy3 = pp3
            
            pp1 = x1, y1
            pp2 = x2, y2
            pp3 = x3, y3
            
#           diferencia 
            d1 = math.sqrt((x1-xx1)**2+(y1-yy1)**2)
            d2 = math.sqrt((x2-xx2)**2+(y2-yy2)**2)
            d3 = math.sqrt((x3-xx3)**2+(y3-yy3)**2)
#             print(d1,d2,d3)
            
            if d1 < 30 and d2 < 30 and d3 < 30:
                if draw:
                    cv.line(img, ((x1,y1)), ((x2,y2)), (255,255,255), 2)
                    cv.line(img, ((x2,y2)), ((x3,y3)), (255,255,255), 2)
#             
                    cv.circle(img, (x1,y1), 5, (0,0,255), cv.FILLED)
                    cv.circle(img, (x1,y1), 10, (0,0,255), 2)
                    cv.circle(img, (x2,y2), 5, (0,0,255), cv.FILLED)
                    cv.circle(img, (x2,y2), 10, (0,0,255), 2)
                    cv.circle(img, (x3,y3), 5, (0,0,255), cv.FILLED)
                    cv.circle(img, (x3,y3), 10, (0,0,255), 2)
        
        return angle,pp1,pp2,pp3
                
    
    def HandsUp(self, img, ang1, ang2, p1, p2, p3, draw = True):

        x1,y1 = self.lmList[p1][1:]
        x2,y2 = self.lmList[p2][1:]
        x3,y3 = self.lmList[p3][1:]
        handsUp = False
        
        if (ang1 and ang2) != None:
            if ang1 > 40 or ang2 > 40:
                cv.putText(img, 'guardia arriba!', (200, 50), cv.FONT_HERSHEY_PLAIN, 2, (0,0,255), 3)
            else:
                handsUp = True
        return handsUp
    
    
    def Bag(self, img, md, mi, draw = True):
        
        cv.putText(img, 'pouch!', (400, 50), cv.FONT_HERSHEY_PLAIN, 2, (0,0,0), 3)
        cv.putText(img, 'pouch!', (400, 50), cv.FONT_HERSHEY_PLAIN, 2, (255,255,255), 2)
        cv.rectangle(img, (50, 75), (150, 400),(0,0,255), -1)
        cv.rectangle(img, (60, 400), (140, 420),(0,0,0), -1)
        cv.rectangle(img, (60,55), (140, 75), (0,0,0), -1)
        cv.line(img, (60,55), (100,0), (0,0,0), 2)
        cv.line(img, (140,55), (100,0), (0,0,0), 2)
        xd,yd = self.lmList[md][1:]
        
        xi,yi = self.lmList[mi][1:]
# #       TIMER      
#         then = datetime.datetime.now() + datetime.timedelta(seconds=10)
#         while then > datetime.datetime.now():
#             mili = mili + 1
#             if mili == 100:
#                 seg = seg + 1
#                 print (seg + ':' + mili)
#             time.sleep(0.01)
    
        if xd < 140:
            cv.line(img, (xd+10,yd+10), (xd+30,yd+30), (255,255,255), 2)
            cv.line(img, (xd+10,yd-10), (xd+30,yd-30), (255,255,255), 2)
            cv.line(img, (xd+10,yd), (xd+30,yd), (255,255,255), 2)
            cv.line(img, (xd+10,yd+10), (xd+30,yd+30), (0,0,0), 1)
            cv.line(img, (xd+10,yd-10), (xd+30,yd-30), (0,0,0), 1)
            cv.line(img, (xd+10,yd), (xd+30,yd), (0,0,0), 1)
        if xi < 140:
            cv.line(img, (xi+10,yi+10), (xi+30,yi+30), (255,255,255), 2)
            cv.line(img, (xi+10,yi-10), (xi+30,yi-30), (255,255,255), 2)
            cv.line(img, (xi+10,yi), (xi+30,yi), (255,255,255), 2)
            cv.line(img, (xi+10,yi+10), (xi+30,yi+30), (0,0,0), 1)
            cv.line(img, (xi+10,yi-10), (xi+30,yi-30), (0,0,0), 1)
            cv.line(img, (xi+10,yi), (xi+30,yi), (0,0,0), 1)
            

            
def Marker(img, markerSize=6, totalMarkers=250, draw=True):

        imgGray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
        key = getattr(aruco, f'DICT_6X6_250')#{markerSize}X{markerSize}_{totalMarkers}')
        arucoDict = aruco.Dictionary_get(key)
        arucoParam = aruco.DetectorParameters_create()
        bboxs, ids, rejected = aruco.detectMarkers(imgGray, arucoDict, parameters=arucoParam)
        try:
            if ids != None:
                print(ids)  
                x1 = int(bboxs[0][0][0][0])
                y1 = int(bboxs[0][0][0][1])
                x2 = int(bboxs[0][0][1][0])
                y2 = int(bboxs[0][0][1][1]) 
                p = abs(x2-x1)
                D = (0.058*0.1*413)/(p*0.03)
#                 D = "%.2f" % round(D,2)
                print(D)
            
                if draw:
                    aruco.drawDetectedMarkers(img, bboxs)
                    cv.rectangle(img, (400, 410),(540,455), (255,255,255), -1 )
                    cv.rectangle(img, (400, 410),(540,455), (0,0,0), 1 )
                    cv.putText(img, 'Distancia: ' + str(D), (402,450), cv.FONT_HERSHEY_PLAIN, 1, (0,0,0), 2)
        except:
            pass

        
def main():
    cap = cv.VideoCapture(0)
    pTime = 0
    cTime = 0
    detector = poseDetector()
    while True:
        success, img = cap.read()
        if not success:
            print('Camara no detectada')
            break
        img = detector.findPose(img)
#         img = cv.resize(img, (1000,800), interpolation=cv.INTER_CUBIC)
        lmList = detector.findPosition(img, draw=False)
       
        
        if len(lmList) != 0:
            try:
                angle1, pa1, pa2, pa3 = detector.findAngle(img, 12, 14, 16, pa1, pa2, pa3)
                angle2, pa4, pa5, pa6 = detector.findAngle(img, 11, 13, 15, pa4, pa5, pa6)
                
#                 grafico grados
#                 cv.rectangle(img, (400, 410),(540,455), (255,255,255), -1 )
#                 cv.rectangle(img, (400, 410),(540,455), (0,0,0), 1 )
#                 cv.putText(img, 'grados der: ' + str(int(angle1)), (402,450), cv.FONT_HERSHEY_PLAIN, 1, (0,0,0), 2)
#                 cv.putText(img, 'grados izq: ' + str(int(angle2)), (402,425), cv.FONT_HERSHEY_PLAIN, 1, (0,0,0), 2)

#                 Manos arriba
#                 detector.HandsUp(img, angle1, angle2, 12, 14, 16)
#                 detector.HandsUp(img, angle1, angle2, 11, 13, 15)
#                 detector.Bag(img, 20, 21)

                
            except:
                pa1 = lmList[12][1:]
                pa2 = lmList[14][1:]
                pa3 = lmList[16][1:]
                pa4 = lmList[12][1:]
                pa5 = lmList[14][1:]
                pa6 = lmList[16][1:]
                pass
            
#       Calculo FPS   
        cTime = time.time()
        fps = 1/(cTime - pTime)
        pTime = cTime
        Marker(img)
        cv.putText(img, 'fps: ' + str(int(fps)), (10,450), cv.FONT_HERSHEY_PLAIN, 1, (255,255,255), 2)
        cv.putText(img, 'fps: ' + str(int(fps)), (10,450), cv.FONT_HERSHEY_PLAIN, 1, (0,0,0), 1)
        cv.imshow("Image", img)
        
        if cv.waitKey(20) & 0xFF==ord('d'):
                 break

    cap.release()
    cv.destroyAllWindows()
        
    
if __name__== "__main__":
    main()